In [21]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>My Crypto Wallet</title>
    <link rel="stylesheet" href="styles.css"> <!-- Link to CSS File -->
</head>
<body>
    <div class="wallet-container">
        <h2>My Crypto Wallet</h2>
        <p><strong>Bitcoin Address:</strong> <a href="" id="address" target="_blank">Generating...</a></p>
        <div class="qr-code">
            <canvas id="qr-code"></canvas>
        </div>
        <div class="price-container" id="crypto-prices">
            <p>Loading cryptocurrency prices...</p>
        </div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/elliptic/6.5.4/elliptic.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/crypto-js/4.1.1/crypto-js.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/qrcode/1.4.4/qrcode.min.js"></script>
    <script src="script.js"></script> <!-- Link to JavaScript File -->
</body>
</html>


Overwriting index.html


In [22]:
%%writefile styles.css
/* styles.css */

body {
    font-family: Arial, sans-serif;
    text-align: center;
    padding: 30px;
    background-color: #f4f4f4;
}

h2 {
    color: #333;
}

.wallet-container {
    display: flex;
    justify-content: center;
    flex-direction: column;
    align-items: center;
}

.wallet-container p {
    font-size: 18px;
}

.qr-code {
    margin-top: 20px;
}

.price-container {
    margin-top: 40px;
    background-color: #333;
    color: white;
    padding: 10px;
    border-radius: 5px;
}


Overwriting styles.css


In [23]:
%%writefile script.js
// script.js

document.addEventListener("DOMContentLoaded", async () => {
    // Initialize elliptic library for key generation (SECP256k1 curve)
    const EC = elliptic.ec;
    const ec = new EC('secp256k1');
    const keyPair = ec.genKeyPair();

    // Generate the private key in hexadecimal format
    const privateKey = keyPair.getPrivate('hex');
    // Generate the public key in hexadecimal format
    const publicKey = keyPair.getPublic('hex');

    // Generate the Bitcoin address from the public key
    const sha256 = CryptoJS.SHA256(CryptoJS.enc.Hex.parse(publicKey));
    const ripemd160 = CryptoJS.RIPEMD160(sha256);
    const address = ripemd160.toString(CryptoJS.enc.Base58);

    // Set the Bitcoin address as a clickable link
    const addressLink = `https://www.coingecko.com/en/coins/bitcoin`; // URL to the Bitcoin page on CoinGecko
    document.getElementById("address").textContent = address;
    document.getElementById("address").setAttribute("href", addressLink);

    // Generate the QR code for the Bitcoin address
    QRCode.toCanvas(document.getElementById("qr-code"), address, {
        width: 150,
    }, function(error) {
        if (error) console.error(error);
        console.log('QR code generated!');
    });

    // Fetch live cryptocurrency prices from a public API
    const cryptoPrices = await fetch("https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum&vs_currencies=usd")
        .then(response => response.json())
        .catch(error => console.error('Error fetching prices:', error));

    // Display the fetched cryptocurrency prices
    const bitcoinPrice = cryptoPrices?.bitcoin?.usd || "N/A";
    const ethereumPrice = cryptoPrices?.ethereum?.usd || "N/A";

    document.getElementById("crypto-prices").innerHTML = `
        <p><strong>Bitcoin (BTC):</strong> $${bitcoinPrice}</p>
        <p><strong>Ethereum (ETH):</strong> $${ethereumPrice}</p>
    `;
});


Overwriting script.js


In [15]:
!pip install ecdsa